In [1]:
import pandas as pd
import glob
import os
import shutil
import re
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [2]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [3]:
dataset_names = ['Dataset011_5FoldCV', 'Dataset012_5FoldCV', 'Dataset013_5FoldCV', 'Dataset014_5FoldCV', 'Dataset015_5FoldCV']

In [4]:
preds = glob.glob('/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset01*/test*/*.nii.gz')
len(preds)

154

In [5]:
preds[0]

'/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset015_5FoldCV/test_fold1/VentrikelCNN_083.nii.gz'

In [6]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: x.replace('test_fold1', 'labelsTs'))
#df['fold'] = df.prediction.apply(lambda x: x.split('/')[-2].split('_')[-1])
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [7]:
df.head()

,prediction,gt,ds
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,Dataset015_5FoldCV
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,Dataset015_5FoldCV
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,Dataset015_5FoldCV
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,Dataset015_5FoldCV
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,Dataset015_5FoldCV


In [8]:
df.prediction[0]

'/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset015_5FoldCV/test_fold1/VentrikelCNN_083.nii.gz'

In [9]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_dice_n(i, j, [1,2,3,4,5]))

/media/raid/orco/data/ICH/code/neurorad_bids_utils/bids_utils/utils/utils.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return 2*intersection/(a_arr.sum()+b_arr.sum())


In [10]:
df = pd.DataFrame(res)
df.head()

,0,1,2,3,4,5,6
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.945415,0.955518,0.969490,0.963869,0.845411
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.981807,0.984666,0.965899,0.948001,0.816176
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.979531,0.961349,0.955765,0.766721,0.392157
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.944871,0.944085,0.834331,0.920129,0.795181
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.891487,0.974966,0.931481,0.926316,0.000000


In [11]:
len(df.columns)

7

In [12]:
df.columns = ['prediction', 'gt', 'DSC_left', 'DSC_right', 'DSC_3rd', 'DSC_4th', 'DSC_Acq']


In [13]:
#df['fold'] = df.prediction.apply(lambda x: x.split('/')[-2].split('_')[-1])
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [14]:
df.head()

,prediction,gt,DSC_left,DSC_right,DSC_3rd,DSC_4th,DSC_Acq,ds
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.945415,0.955518,0.969490,0.963869,0.845411,Dataset015_5FoldCV
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.981807,0.984666,0.965899,0.948001,0.816176,Dataset015_5FoldCV
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.979531,0.961349,0.955765,0.766721,0.392157,Dataset015_5FoldCV
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.944871,0.944085,0.834331,0.920129,0.795181,Dataset015_5FoldCV
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.891487,0.974966,0.931481,0.926316,0.000000,Dataset015_5FoldCV


In [21]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [22]:
df.head()

,prediction,gt,DSC_left,DSC_right,DSC_3rd,DSC_4th,DSC_Acq,ds,sid
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.945415,0.955518,0.969490,0.963869,0.845411,Dataset015_5FoldCV,083
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.981807,0.984666,0.965899,0.948001,0.816176,Dataset015_5FoldCV,108
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.979531,0.961349,0.955765,0.766721,0.392157,Dataset015_5FoldCV,115
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.944871,0.944085,0.834331,0.920129,0.795181,Dataset015_5FoldCV,074
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.891487,0.974966,0.931481,0.926316,0.000000,Dataset015_5FoldCV,266


In [24]:
df = df[['sid','ds'] + [i for i in df.columns if i.startswith('DSC')] + ['gt', 'prediction']]

In [27]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

'2024319'

In [30]:
df.to_csv('./data/DSC_nnUNet_5classes_{}_{}.csv'.format('5FoldCV', date), index=False)